In [1]:
! pip install  torch_geometric==2.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.7 MB/s eta 0:00:00


# Import Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random
import numpy as np
from scipy.sparse import coo_matrix
from torch_geometric.nn import GATv2Conv
from sklearn.cluster import KMeans
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj
import torch_geometric.utils as pyg_utils
import networkx as nx
import scipy.sparse as sp
from torch_geometric.data import Data
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import f1_score as F1
from sklearn.metrics import adjusted_rand_score

# Utility Functions

In [3]:

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
def relu(x: np.array):
    x = (np.abs(x) + x) / 2.0
    return x


def ONMI(X, Y):
    """
        Compute NMI between two overlapping community covers.

        Parameters
        ----------
        X : array-like, shape [N, m]
            Matrix with samples stored as columns.
        Y : array-like, shape [N, n]
            Matrix with samples stored as columns.

        Returns
        -------
        nmi : float
            Float in [0, 1] quantifying the agreement between the two partitions.
            Higher is better.

        References
        ----------
        McDaid, Aaron F., Derek Greene, and Neil Hurley.
        "Normalized mutual information to evaluate overlapping
        community finding algorithms."
        arXiv preprint arXiv:1110.2515 (2011).

    """
    if not ((X == 0) | (X == 1)).all():
        raise ValueError("X should be a binary matrix")
    if not ((Y == 0) | (Y == 1)).all():
        raise ValueError("Y should be a binary matrix")

    if X.shape[1] > X.shape[0] or Y.shape[1] > Y.shape[0]:
        warnings.warn("It seems that you forgot to transpose the F matrix")
    X = X.T
    Y = Y.T

    def cmp(x, y):
        """Compare two binary vectors."""
        a = (1 - x).dot(1 - y)
        d = x.dot(y)
        c = (1 - y).dot(x)
        b = (1 - x).dot(y)
        return a, b, c, d

    def h(w, n):
        """Compute contribution of a single term to the entropy."""
        if w == 0:
            return 0
        else:
            return -w * np.log2(w / n)

    def H(x, y):
        """Compute conditional entropy between two vectors."""
        a, b, c, d = cmp(x, y)
        n = len(x)
        if h(a, n) + h(d, n) >= h(b, n) + h(c, n):
            return h(a, n) + h(b, n) + h(c, n) + h(d, n) - h(b + d, n) - h(a + c, n)
        else:
            return h(c + d, n) + h(a + b, n)

    def H_uncond(X):
        """Compute unconditional entropy of a single binary matrix."""
        return sum(h(x.sum(), len(x)) + h(len(x) - x.sum(), len(x)) for x in X)

    def H_cond(X, Y):
        """Compute conditional entropy between two binary matrices."""
        m, n = X.shape[0], Y.shape[0]
        scores = np.zeros([m, n])
        for i in range(m):
            for j in range(n):
                scores[i, j] = H(X[i], Y[j])
        return scores.min(axis=1).sum()

    if X.shape[1] != Y.shape[1]:
        raise ValueError("Dimensions of X and Y don't match. (Samples must be stored as COLUMNS)")
    H_X = H_uncond(X)
    H_Y = H_uncond(Y)
    I_XY = 0.5 * (H_X + H_Y - H_cond(X, Y) - H_cond(Y, X))
    return I_XY / max(H_X, H_Y)


def find_best_thresh(label: np.array, z: np.array):
    """
    寻找 overlapping_nmi最大时的阈值
    :param label: 重叠社区标签， np.array  [N * K]
    :param z: 社区表示矩阵， np.array  [N * K]
    :return:

    """
    thresh = 0.
    best_thresh = 0.
    max_onmi = 0.
    while thresh <= np.max(z):
        onmi_val = overlapping_nmi(label, z, thresh)
        if onmi_val >= max_onmi:
            best_thresh = thresh
            max_onmi = onmi_val
        thresh += 0.01
    return best_thresh



def overlapping_nmi(label: np.array, z: np.array, thresh=0.5):

    z = relu(z)
    pred = z > thresh
    return ONMI(label, pred)


def overlapping_f1_score(label: np.array, z: np.array, thresh=0.5):

    pred = z > thresh
    return F1(label, pred, average="micro")

def load_pyg_dataset(file_name, train_size, p_mis):
    if not file_name.endswith('.npz'):
        file_name += '.npz'
    with np.load(file_name, allow_pickle=True) as loader:
        loader = dict(loader)
        A = sp.csr_matrix((loader['adj_matrix.data'], loader['adj_matrix.indices'],
                           loader['adj_matrix.indptr']), shape=loader['adj_matrix.shape'])

        if 'attr_matrix.data' in loader.keys():
            X = sp.csr_matrix((loader['attr_matrix.data'], loader['attr_matrix.indices'],
                               loader['attr_matrix.indptr']), shape=loader['attr_matrix.shape'])
        else:
            X = None

        Z = sp.csr_matrix((loader['labels.data'], loader['labels.indices'],
                           loader['labels.indptr']), shape=loader['labels.shape'])

        # Remove self-loops
        A = A.tolil()
        A.setdiag(0)
        A = A.tocsr()

        # Convert label matrix to numpy
        if sp.issparse(Z):
            Z = Z.toarray().astype(np.float32)

        # Convert data to PyTorch tensors
        x = torch.tensor(X.toarray() if X is not None else [], dtype=torch.float)
        edge_index = torch.tensor(A.nonzero(), dtype=torch.long)
        y = torch.tensor(Z, dtype=torch.long)
        num_classes = y.shape[1]
        # print('num classes:', num_classes)
        
        num_nodes = x.shape[0]
        num_train_nodes = int(train_size * num_nodes)

        # Create a shuffled array of indices
        indices = np.random.permutation(num_nodes)

        # Create masks
        train_mask = torch.zeros(num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(num_nodes, dtype=torch.bool)

        # Assign indices to the training and test sets
        train_mask[indices[:num_train_nodes]] = True
        test_mask[indices[num_train_nodes:]] = True

        # Feature swapping based on p_mis
        if( p_mis>0.0):
            # print(p_mis*100)
            swap_count = int(num_nodes * p_mis / 2)
            for _ in range(swap_count):
                a = random.randint(0, num_nodes - 1)
                b = random.randint(0, num_nodes - 1)
                #print(a, ' ', b)
                if a != b:
                    x[[a, b], :] = x[[b, a], :]

        # Create PyTorch Geometric Data object
        data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)
        data.num_classes = num_classes
        data.file_name = file_name
        data.train_size = train_size

        return data



# **Define The Module**

In [4]:
class Encoder(nn.Module):
    def __init__(self, in_features, hidden_dim, latent_dim, num_heads, num_class):
        super(Encoder, self).__init__()
        self.gat_layer = GATv2Conv(
            in_channels=in_features,
            out_channels=hidden_dim,
            heads=num_heads,
        )
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim * num_heads)
        
        self.gat_layer2 = GATv2Conv(
            in_channels=hidden_dim * num_heads,
            out_channels=latent_dim,
            heads=num_heads,
        )
        self.batch_norm2 = nn.BatchNorm1d(latent_dim * num_heads)
        
        self.fc = nn.Linear(latent_dim * num_heads, num_class)


    def forward(self, x, edge_index):
        x = self.gat_layer(x, edge_index)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = self.batch_norm1(x)
        x = F.dropout(x, p=0.2, training=self.training)
        
        x = self.gat_layer2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = F.dropout(x, p=0.2, training=self.training)

        x = self.fc(x)
        m = F.normalize(x, p=2, dim=1)
        x = F.softmax(x, dim=1)
        a_bar = torch.sigmoid(torch.matmul(m, m.t()))

        return m, x, a_bar

# **Load Dataset**

In [5]:
# Set random seed for reproducibility
file_name = '/kaggle/input/overlappingdata/data/mag_cs.npz'
dataset = load_pyg_dataset(file_name,train_size=0.1, p_mis=0.0) 
print(dataset)

Data(x=[21957, 7793], edge_index=[2, 193500], y=[21957, 18], train_mask=[21957], test_mask=[21957], num_classes=18, file_name='/kaggle/input/overlappingdata/data/mag_cs.npz', train_size=0.1)


/tmp/ipykernel_19/3093907754.py:154: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  edge_index = torch.tensor(A.nonzero(), dtype=torch.long)


In [6]:
#Semi-Supervised Graph Autoencoder for Overlapping Semantic Community Detection

class SSGOCD(nn.Module):
    def __init__(self, dataset, num_class, lr=5e-4, normalize_feature=True, lamda=0.5, alpha=1e-8, beta=1e-6, device='cpu'):
        super(SSGOCD, self).__init__()
        self.dataset = dataset
        self.num_class = num_class
        self.lr = lr
        self.normalize_feature = normalize_feature
        self.lamda = lamda
        self.alpha = alpha
        self.beta = beta
        self.device = device
        
        self.num_nodes = dataset.num_nodes
        self.in_feat = dataset.num_features
        self.n_hidden = 64
        self.latent = 16
        self.n_head = 8
        
        self.B = self.compute_B_matrix()
        self.adj = self.compute_adj_matrix()
        
    def compute_B_matrix(self):
        edge_index = self.dataset.edge_index.cpu().numpy()
        num_nodes = self.dataset.num_nodes
        A = coo_matrix((np.ones(edge_index.shape[1]), (edge_index[0], edge_index[1])), shape=(num_nodes, num_nodes))
        A = torch.tensor(A.toarray(), dtype=torch.float).to(self.device)
        
        D = torch.diag(torch.sum(A, dim=1))
        B = A - (1.0 / A.sum()) * torch.matmul(D, A)
        return B

    def compute_adj_matrix(self):
        edge_index = self.dataset.edge_index.cpu().numpy()
        num_nodes = self.dataset.num_nodes
        A = coo_matrix((np.ones(edge_index.shape[1]), (edge_index[0], edge_index[1])), shape=(num_nodes, num_nodes))
        A = torch.tensor(A.toarray(), dtype=torch.float).to(self.device)
        return A

    def l2_reg_loss(self, model, scale=1e-4):
        loss = 0.0
        for w in model.parameters():
            loss += w.pow(2.).sum()
        return loss * scale

    def l1_reg_loss(self, model, scale=1e-4):
        loss = 0.0
        for w in model.parameters():
            loss += w.abs().sum()
        return loss * scale

    def train(self, max_epoch=20, unsupervised=False, use_modularity=True):
        self.to(self.device)
        data = self.dataset.to(self.device)

        if self.normalize_feature:
            data.x = F.normalize(data.x)

        encoder = Encoder(self.in_feat, self.n_hidden, self.latent, self.n_head, self.num_class).to(self.device)
        optimizer = optim.AdamW(encoder.parameters(), lr=self.lr, weight_decay=5e-4)
        
        train_mask = data.train_mask
        test_mask = data.test_mask
        labels = data.y
        
        nmi_results = []
        f1_score_results = []
        
        for epoch in range(1, max_epoch + 1):
            encoder.train()
            optimizer.zero_grad()
            
            m, z, A_bar = encoder(data.x, data.edge_index)
            loss = F.mse_loss(A_bar, self.adj)

            if not unsupervised:
                loss += self.lamda * F.cross_entropy(z[train_mask], labels[train_mask].float())

            if use_modularity:
                modularity_loss = self.beta * torch.trace(m.t().matmul(self.B).matmul(m))
                loss -= modularity_loss
            
            loss += self.l2_reg_loss(encoder, 1e-5)
            loss += self.l1_reg_loss(encoder, 1e-6)

            loss.backward()
            optimizer.step()

            if epoch % 1 == 0:
               # self.eval()
                thresh = 3.0 / self.num_class
                nmi = overlapping_nmi(labels[test_mask].cpu().numpy(), z[test_mask].detach().cpu().numpy(), thresh)
                f1_score = overlapping_f1_score(labels[test_mask].cpu().numpy(), z[test_mask].detach().cpu().numpy(), thresh)

                nmi_results.append(nmi)
                f1_score_results.append(f1_score)
                
                if epoch % 10 == 0:
                    print(f'Epoch {epoch}, Loss: {loss.item():.4f}, NMI: {nmi:.4f}, F1 Score: {f1_score:.4f}')

        print(f'Max Overlapping NMI: {np.max(nmi_results):.4f}')
        print(f'Max Overlapping F1 Score: {np.max(f1_score_results):.4f}')



# Initialize and train model

In [7]:
print(dataset.train_mask.sum())
print(dataset.test_mask.sum())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SSGOCD(dataset, num_class=dataset.num_classes, lr=0.006, lamda=0.5, alpha=1e-5, beta=1e-6, device=device)
model.train(max_epoch=250, unsupervised=False)

tensor(2195)
tensor(19762)
Epoch 10, Loss: 1.8745, NMI: 0.4505, F1 Score: 0.7546
Epoch 20, Loss: 1.7873, NMI: 0.5041, F1 Score: 0.7908
Epoch 30, Loss: 1.7393, NMI: 0.5687, F1 Score: 0.8223
Epoch 40, Loss: 1.7162, NMI: 0.5637, F1 Score: 0.8203
Epoch 50, Loss: 1.7009, NMI: 0.5622, F1 Score: 0.8194
Epoch 60, Loss: 1.6916, NMI: 0.5623, F1 Score: 0.8194
Epoch 70, Loss: 1.6846, NMI: 0.5620, F1 Score: 0.8190
Epoch 80, Loss: 1.6798, NMI: 0.5600, F1 Score: 0.8178
Epoch 90, Loss: 1.6768, NMI: 0.5563, F1 Score: 0.8153
Epoch 100, Loss: 1.6744, NMI: 0.5533, F1 Score: 0.8134
Epoch 110, Loss: 1.6725, NMI: 0.5493, F1 Score: 0.8107
Epoch 120, Loss: 1.6717, NMI: 0.5482, F1 Score: 0.8103
Epoch 130, Loss: 1.6703, NMI: 0.5487, F1 Score: 0.8105
Epoch 140, Loss: 1.6693, NMI: 0.5426, F1 Score: 0.8058
Epoch 150, Loss: 1.6690, NMI: 0.5469, F1 Score: 0.8096
Epoch 160, Loss: 1.6674, NMI: 0.5440, F1 Score: 0.8072
Epoch 170, Loss: 1.6667, NMI: 0.5430, F1 Score: 0.8066
Epoch 180, Loss: 1.6668, NMI: 0.5411, F1 Score: